All necessary imports

In [1]:
import requests
import json
import arcpy
import os
import sys
import time
from arcpy.sa import *

Fetching the data from USGS server via OpenTopography API

In [ ]:
import requests

# Define the bounding box and dataset
bbox = "-105.3,39.9,-105.2,40.1"  # minX,minY,maxX,maxY
datasets = "National Elevation Dataset (NED) 1 arc-second Current"

# Construct the API request URL
api_url = f"https://tnmaccess.nationalmap.gov/api/v1/products?bbox={bbox}&datasets={datasets}&outputFormat=JSON"

# Send the request
response = requests.get(api_url)

# Check if the request was successful
if response.status_code == 200:
    data = response.json()
    # Process the data as needed
    for item in data.get('items', []):
        print(f"Product: {item.get('title')}")
        print(f"Download URL: {item.get('downloadURL')}\n")
else:
    print(f"Request failed with status code {response.status_code}")


Now that we have code to download specific bounding boxes, we initiate arcGIS environment.

In [ ]:

# Set environment
arcpy.env.workspace = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1"
arcpy.env.overwriteOutput = True

# Input DEM
input_dem = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\merged_dem.tif"

# Set output path
filled_dem = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\filled_dem.tif"

# Execute Fill
arcpy.CheckOutExtension("Spatial")
filled_raster = Fill(input_dem)
filled_raster.save(filled_dem)

print("Sink filling completed. Filled DEM saved at:", filled_dem)


Sink filling completed. Filled DEM saved at: C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\filled_dem.tif


Now we find the flow direction raster

Output: Flow Direction raster

In [ ]:

flow_direction_raster = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\flow_direction.tif"

# Execute Flow Direction
arcpy.CheckOutExtension("Spatial")
flow_dir = FlowDirection(filled_dem, "NORMAL")  # Use "NORMAL" for standard flow direction
flow_dir.save(flow_direction_raster)

print("Flow direction calculation completed. Output saved at:", flow_direction_raster)


Flow direction calculation completed. Output saved at: C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\flow_direction.tif


Now we find the flow accumulation raster

Output: Flow Accumulation raster

In [ ]:

flow_accumulation_raster = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\flow_accumulation.tif"

# Execute Flow Accumulation
arcpy.CheckOutExtension("Spatial")
flow_acc = FlowAccumulation(flow_direction_raster)
flow_acc.save(flow_accumulation_raster)

print("Flow accumulation calculation completed. Output saved at:", flow_accumulation_raster)

Flow accumulation calculation completed. Output saved at: C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\flow_accumulation.tif


CRITICAL TO ACCURACY PART

WE CHANGE THE SYMBOLOGY TO BINARY CLASSES AND FIND THE RESOLUTION AT WHICH OUR RIVERS ARE CLEARLY VISIBLE

In [ ]:

flow_accum_raster = Raster(flow_accumulation_raster)

# Define output path
binary_stream_raster = r"C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\binary_stream.tif"

# Define threshold
stream_threshold = 20000  # Change this to test different stream definitions

# Apply threshold to create binary raster
binary_stream = Con(flow_accum_raster > stream_threshold, 1, 0)
binary_stream.save(binary_stream_raster)

print(f"Binary stream raster created with threshold {stream_threshold}. Output saved at:", binary_stream_raster)

Binary stream raster created with threshold 20000. Output saved at: C:\Users\wishu\Documents\ArcGIS\Projects\NYUTask1\binary_stream.tif


Now that we have a clear map of river streams 

Save the shapefiles containing outlet points

Proceed with code in next cell to produce desired results

In [ ]:
outlet_shapefile = input("C:/path/to/output/watershed.shp").strip()

# Output: Watershed raster
watershed_raster = "C:/path/to/output/watershed.tif"

# Ensure the Spatial Analyst extension is checked out
arcpy.CheckOutExtension("Spatial")

# Execute Watershed delineation
watershed = Watershed(flow_direction_raster, outlet_shapefile)
watershed.save(watershed_raster)

print(f"Watershed delineation completed. Output saved at: {watershed_raster}")